In [0]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/My \Drive/Sem \8 \files/
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/Sem 8 files
 data				  __pycache__
 euclidean_distances.json	  Shortest_path_lengths.pkl
'Fri Feb 14 09_06_16 2020.json'   utils.py


In [0]:
!apt-get install -y libspatialindex-c4v5
!pip3.6 install osmnx

import osmnx as ox
import networkx as nx
import pandas as pd
import json
import random
import statistics
import numpy as np

from rtree import index
from utils import *

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 270 kB of archives.
After this operation, 1,107 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Fetched 270 kB in 1s (366 kB/s)
Selecting previously unselected package libspatialindex4v5:amd64.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../libspatialindex4v5_1.8.5-5_amd64.deb ...
Unpacking libspatialindex4v5:amd64 (1.8.5-5) ...
Selecting previously unselected package libspatialindex-c4v5:amd64.
Preparing to u

In [0]:
place_name = "Singapore, Central, Singapore"
G = ox.save_load.load_graphml(filename="Singapore_drive_processed.graphml")

# Stores nodes and edges mapped with speed bands along with other attributes
type(G)

networkx.classes.multidigraph.MultiDiGraph

In [0]:
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
nodes = pd.read_pickle('data/nodes_drive.pkl')

In [0]:
f = open("Fri Feb 14 09_06_16 2020.json", "r").read()
x = json.loads(f)

speed_bands = pd.DataFrame.from_dict(x, orient='columns')
process_speed_band(speed_bands)

/content/drive/My Drive/Sem 8 files/utils.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MaximumSpeed'][i] = '80'


In [0]:
calculate_congestion(G, edges, speed_bands)

In [0]:
# Filter out all bus stop nodes
# key: node label ,value: location
idx = index.Index()

xx = nodes['x'].astype(float)
yy = nodes['y'].astype(float)
node_id = nodes['osmid']
node_type = nodes['highway']

# nan = 18823

# speed_camera = 2
# motorway_junction = 222
# turning_circle = 79
# crossing = 27
# turning_loop = 26
# mini_roundabout = 6
# traffic signals = 4031

for node, x, y, kind in zip(node_id, xx, yy, node_type):
    if type(kind) == float:
      idx.insert(int(node), (x,y,x,y))

### Calculate Variance

In [0]:
def find_nearest_nodes(nodes, source, dist, idx):
    # Create a bounding box around source of min distance in all directions
    (north, south, east, west) = ox.bbox_from_point(point=source, distance=dist)
    
    candidate_nodes = []
    initial = list(idx.intersection((west, south, east, north)))   
    for node in initial:
      x = haversine_distance(nodes, node, source)
      if x <= dist:
        candidate_nodes.append(node)

    return candidate_nodes

In [0]:
def get_length_dict(sources, destinations, method='euclidean'):
    distances = dict()
    
    if method == 'euclidean':
        for source in sources:        
            for dest in destinations:
                ed = haversine_distance(nodes, source, dest)
                distances[source,dest] = ed/13.8889
    else:
        for source in sources:        
            for dest in destinations:
                try:
                    # dijsktra distance in sec
                    d = nx.shortest_path_length(G, source, dest, weight='BPR', method='dijkstra')
                except nx.NetworkXUnfeasible:
                    d = float('inf')
                distances[(source,dest)] = d
       
    # sort the dictionary in ascending order using distances
    x = sorted(distances.items(), key = lambda kv:(kv[1], kv[0]))
    return x    

In [0]:
def hybrid_search_count(G, x):
    n = len(x)
    min_pair = x[0]
    source = min_pair[0][0]
    dest = min_pair[0][1]
    min_tt = float('inf')
    final_route = []
    count = 0

    for i in range(n):
        try:            
            travel_time = nx.shortest_path_length(G, source, dest, weight='BPR', method='dijkstra')
            count = count + 1
        except nx.NetworkXUnfeasible:
            continue

        if travel_time < min_tt:
            min_tt = travel_time
            min_pair = x[i]

            if i < n-1:
                if travel_time < x[i+1][1]:
                    break
                else:
                    source = x[i+1][0][0]
                    dest = x[i+1][0][1]
            else:
                break
        else:
            if i < n-1:
                if min_tt < x[i+1][1]:
                    break
                else:
                    source = x[i+1][0][0]
                    dest = x[i+1][0][1]
            else:
                break

    return count, min_tt

In [0]:
def lets_do_this():
  # get 10 random sources
  sources = []
  for i in range(20):
      r = random.randint(0,23210)
      s = nodes['osmid'].values[r]
      sources.append(s)
      
  # get 10 correspondig random targets
  targets = []
  for i in range(20):
      r = random.randint(0,23210)
      t = nodes['osmid'].values[r]
      targets.append(t)

  exhaustive = []
  hybrid = []
  for s,t in zip(sources,targets):
      orig_point = nodes[nodes['osmid'] == s].geometry.values[0]
      target_point = nodes[nodes['osmid'] == t].geometry.values[0]
      orig_xy = (orig_point.y, orig_point.x)
      target_xy = (target_point.y, target_point.x)
      
      # Max distance for walking is 720m (10 min * 1.2 m/s)
      # Find all nodes within 720 m from candidate sources and candidate destinations
      candidate_dest = find_nearest_nodes(nodes, target_xy, 500, idx)
      candidate_source = find_nearest_nodes(nodes, orig_xy, 500, idx)

      if len(candidate_dest) == 0:
        candidate_dest.append(ox.get_nearest_node(G, target_xy))
      
      if len(candidate_source) == 0:
        candidate_source.append(ox.get_nearest_node(G, orig_xy))

      value1 = len(candidate_dest) * len(candidate_source)
      print(value1)
      exhaustive.append(value1)
           
      euclidean = get_length_dict(candidate_source, candidate_dest)
      value2, tt = hybrid_search_count(G, euclidean)
      hybrid.append(value2)

      saved = ((value1 - value2)/value1) * 100
      
      print(saved)
      
  return exhaustive, hybrid

In [0]:
exhaustive, hybrid = lets_do_this()

In [0]:
exhaustive = [2166, 1425, 3150, 340, 2144, 1587, 1575, 2686, 1778, 7072, 5600, 3096, 455, 5967, 1116, 1056, 2380, 812, 3672, 200, 2277, 5396, 442, 1652, 1705, 3740, 1836, 2310, 5452, 6004, 465, 864, 2352, 3009, 7316, 2656, 1920, 729, 280, 858]

In [0]:
hybrid = [2166, 190, 3150, 0, 2144, 225, 1575, 236, 1778, 7072, 172, 3096, 165, 1415, 1116, 1056, 0, 10, 373, 200, 2277, 5396, 2, 1652, 1705, 3740, 1836, 2310, 5452, 5924, 465, 864, 19, 3009, 7316, 2656, 171, 729, 280, 29]

In [100]:
print("Number of queries")
print("Exhaustive || Hybrid || Saved %")
sav = []
for e,h in zip(exhaustive, hybrid):
  saved = ((e-h)/e)*100
  print(e,h,saved)
  sav.append(saved)

Number of queries
Exhaustive || Hybrid || Saved %
2166 2166 0.0
1425 190 86.66666666666667
3150 3150 0.0
340 0 100.0
2144 2144 0.0
1587 225 85.82230623818525
1575 1575 0.0
2686 236 91.21370067014148
1778 1778 0.0
7072 7072 0.0
5600 172 96.92857142857143
3096 3096 0.0
455 165 63.73626373626373
5967 1415 76.28624099212334
1116 1116 0.0
1056 1056 0.0
2380 0 100.0
812 10 98.76847290640394
3672 373 89.84204793028321
200 200 0.0
2277 2277 0.0
5396 5396 0.0
442 2 99.5475113122172
1652 1652 0.0
1705 1705 0.0
3740 3740 0.0
1836 1836 0.0
2310 2310 0.0
5452 5452 0.0
6004 5924 1.3324450366422385
465 465 0.0
864 864 0.0
2352 19 99.1921768707483
3009 3009 0.0
7316 7316 0.0
2656 2656 0.0
1920 171 91.09375
729 729 0.0
280 280 0.0
858 29 96.62004662004662


In [101]:
print("Average")
print("Exhaustive:", statistics.mean(exhaustive))
print("Hybrid:", statistics.mean(hybrid))
print("Saved%:", statistics.mean(sav))

Average
Exhaustive: 2488.5
Hybrid: 1799.275
Saved%: 31.926255010207335
